In [71]:
#from transform import *
from utilities import load_csv_data
import os 

## import the csv file for the EMG

In [2]:
os.getcwd()

'd:\\Programming and learning\\Bioinstrument\\FingerRegress\\biomed-a21\\python\\src\\ai'

In [72]:
dirname = r"acq-3"
file_name_emg = "emg-0.csv"
file_name_hand_angle= "Hand_angles_record_3.csv"
file_path_emg = os.path.join(dirname,file_name_emg)
file_path_angle = os.path.join(".",file_name_hand_angle)
data_emg  = load_csv_data(file_name=file_path_emg)
data_hand_angle  = load_csv_data(file_name=file_name_hand_angle,sep=",",delimiter=",")

In [73]:
data_hand_angle.head()

,timestamp,mcp angle,pip angle,dip angle
0,1.638640e+09,30.044020,16.874167,4.065900
1,1.638640e+09,30.242750,16.904562,5.131242
2,1.638640e+09,29.861519,16.800011,4.258537
3,1.638640e+09,30.575831,17.090059,4.463044
4,1.638640e+09,30.278286,16.910310,3.889203


In [74]:
print(f"Angle measure ->the min time is {data_hand_angle.timestamp.min()} and the max time is {data_hand_angle.timestamp.max()}")

Angle measure ->the min time is 1638639900.053265 and the max time is 1638639932.984208


In [75]:
print(f"EMG measure ->the min time is {data_emg.timestamp.min()} and the max time is {data_emg.timestamp.max()}")

EMG measure ->the min time is 1638639859.641768 and the max time is 1638639942.4269779


### We take the smallest min time as the min time of ref.We do the same for the max time
the start time is from EMG measure and the end time is from Angle Measure

In [76]:

time_recording = data_hand_angle.timestamp.max()-data_hand_angle.timestamp.min()

In [77]:
time_recording

32.93094301223755

In [78]:
#difference between the min and the max for time recording of data_hangle
data_hand_angle.timestamp.max()-data_hand_angle.timestamp.min()

32.93094301223755

In [200]:
start = data_hand_angle.timestamp.min()
end =  data_hand_angle.timestamp.max()

In [203]:
#the start time is from EMG measure and the end time is from Angle Measure
emg_crop = data_emg.loc[(data_emg.timestamp>start) & (data_emg.timestamp<end)]
angle_crop=data_hand_angle[(data_hand_angle.timestamp>start) & (data_hand_angle.timestamp<end)]

In [204]:
emg_crop.reset_index(drop=True, inplace=True)
angle_crop.reset_index(drop=True, inplace=True)

In [206]:
from sklearn.base import BaseEstimator, TransformerMixin
from utilities import *
import numpy as np


class WindowsTransformer(BaseEstimator, TransformerMixin):
    def __init__(self,windows_number,factor):
        self.windows_number = windows_number
        self.factor=factor

    def fit(self, X):
        pass

    def transform(self, X,**kwargs):
    
        result=[]
        len_data = X.shape[0]
        
        windows_size = int(len_data/self.windows_number)
        #each window correspond to an example
        data_windows_ch0 = [X[w*windows_size:w*windows_size+windows_size] for w in range(self.windows_number)]

        result = data_windows_ch0
        return result

In [208]:

class FeaturesTransformEMG(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self,X):
        pass
    def transform(self,X):
        dataset =[]
        func_params_list = [getMAV,getVar,getSD,getSSC,getZC,getRMS]
        for exemple in X:
            chann_1= np.array(exemple)  
            x_1=list()
            for func in func_params_list:
                x_1.append(func(chann_1))
            dataset.append(x_1)
        return dataset

class FeaturesTransformAngle(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self,X):
        pass
    def transform(self,X): 
        return [angle.mean().tolist() for angle in X]

In [209]:
windows_transform = WindowsTransformer(32*10,1)
angle_transformed = windows_transform.transform(angle_crop.drop(columns=["timestamp"]))
emg_transformed = windows_transform.transform(emg_crop.value)

,mcp angle,pip angle,dip angle
0,30.242750,16.904562,5.131242
1,29.861519,16.800011,4.258537


In [195]:
feature_angle_tr = FeaturesTransformAngle()
feature_emg_tr = FeaturesTransformEMG()
target = feature_angle_tr.transform(angle_transformed)
trainset = feature_emg_tr.transform(emg_transformed)

In [217]:
from sklearn.model_selection import train_test_split , cross_val_score ,  RepeatedKFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.metrics import r2_score
from sklearn.feature_selection import SelectKBest, RFE, chi2

In [213]:
trainset = scale(trainset)

In [229]:
X_train, X_test, y_train, y_test = train_test_split(trainset,target, test_size=0.20, random_state=42)
clf_lin_reg = LinearRegression() #add parametrers
clf_lin_elastic = ElasticNet()
classfiers = [clf_lin_elastic,clf_lin_reg]

for clf in classfiers: 
    selector = RFE(clf, n_features_to_select=1).fit(X_train,np.array(y_train)[:,0])
    X_train_transform=selector.transform(X_train)
    print(X_train_transform.shape)
    #X_test_transform=selector.transform(X_test)
    print(f"classifiers {clf.__class__.__name__}")
    print("-"*30)
    clf.fit(X_train_transform,np.array(y_train)[:,0])
    train_score = r2_score(np.array(y_train)[:,0],clf.predict(X_train_transform))
    print(f"Training score {train_score}")

(256, 1)
classifiers ElasticNet
------------------------------
Training score 0.12251045457250864
(256, 1)
classifiers LinearRegression
------------------------------
Training score 0.02183569381272754


In [ ]:
cross_val_score(